In [5]:
import copy
import pickle
from typing import Any, Callable, List, Optional, Tuple
import random
import math
import os

import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

Unexpected exception formatting exception. Falling back to standard exception


c:\Users\barun\Miniconda3\envs\torch-zoo\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")
Traceback (most recent call last):
  File "c:\Users\barun\Miniconda3\envs\torch-zoo\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\barun\AppData\Local\Temp\ipykernel_2784\1451234061.py", line 17, in <module>
    import torchvision
  File "c:\Users\barun\Miniconda3\envs\torch-zoo\lib\site-packages\torchvision\__init__.py", line 5, in <module>
    from torchvision import datasets
  File "c:\Users\barun\Miniconda3\envs\torch-zoo\lib\site-packages\torchvision\datasets\__init__.py", line 1, in <module>
    from ._optical_flow import KittiFlow, Sintel, FlyingChairs, FlyingThings3D, HD1K
  File "c:\Users\barun\Miniconda3\envs\torch-zoo\lib\site-pac

## Some Helper functions (taken from CV Project 2)

In [3]:
def normalize_img(img: np.ndarray) -> np.ndarray:
    """Bring image values to [0,1] range
    Args:
        img: (H,W,C) or (H,W) image
    """
    img -= img.min()
    img /= img.max()
    return img


# def verify(function: Callable) -> str:
#     """Will indicate with a print statement whether assertions passed or failed
#     within function argument call.
#     Args:
#         function: Python function object
#     Returns:
#         string that is colored red or green when printed, indicating success
#     """
#     try:
#         function()
#         return '\x1b[32m"Correct"\x1b[0m'
#     except AssertionError:
#         return '\x1b[31m"Wrong"\x1b[0m'


def rgb2gray(img: np.ndarray) -> np.ndarray:
    """Use the coefficients used in OpenCV, found here:
    https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html
    Args:
        Numpy array of shape (M,N,3) representing RGB image in HWC format
    Returns:
        Numpy array of shape (M,N) representing grayscale image
    """
    # Grayscale coefficients
    c = [0.299, 0.587, 0.114]
    return img[:, :, 0] * c[0] + img[:, :, 1] * c[1] + img[:, :, 2] * c[2]


def PIL_resize(img: np.ndarray, size: Tuple[int, int]) -> np.ndarray:
    """
    Args:
        img: Array representing an image
        size: Tuple representing new desired (width, height)
    Returns:
        img
    """
    img = numpy_arr_to_PIL_image(img, scale_to_255=True)
    img = img.resize(size)
    img = PIL_image_to_numpy_arr(img)
    return img


def PIL_image_to_numpy_arr(img: Image, downscale_by_255: bool = True) -> np.ndarray:
    """
    Args:
        img: PIL Image
        downscale_by_255: whether to divide uint8 values by 255 to normalize
        values to range [0,1]
    Returns:
        img
    """
    img = np.asarray(img)
    img = img.astype(np.float32)
    if downscale_by_255:
        img /= 255
    return img


def im2single(im: np.ndarray) -> np.ndarray:
    """
    Args:
        img: uint8 array of shape (m,n,c) or (m,n) and in range [0,255]
    Returns:
        im: float or double array of identical shape and in range [0,1]
    """
    im = im.astype(np.float32) / 255
    return im


def single2im(im: np.ndarray) -> np.ndarray:
    """
    Args:
        im: float or double array of shape (m,n,c) or (m,n) and in range [0,1]
    Returns:
        im: uint8 array of identical shape and in range [0,255]
    """
    im *= 255
    im = im.astype(np.uint8)
    return im


def numpy_arr_to_PIL_image(img: np.ndarray, scale_to_255: False) -> PIL.Image:
    """
    Args:
        img: in [0,1]
    Returns:
        img in [0,255]
    """
    if scale_to_255:
        img *= 255
    return PIL.Image.fromarray(np.uint8(img))


def load_image(path: str) -> np.ndarray:
    """
    Args:
        path: string representing a file path to an image
    Returns:
        float_img_rgb: float or double array of shape (m,n,c) or (m,n)
           and in range [0,1], representing an RGB image
    """
    img = PIL.Image.open(path)
    img = np.asarray(img, dtype=float)
    float_img_rgb = im2single(img)
    return float_img_rgb


def save_image(path: str, im: np.ndarray) -> None:
    """
    Args:
        path: string representing a file path to an image
        img: numpy array
    """
    folder_path = os.path.split(path)[0]
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    img = copy.deepcopy(im)
    img = single2im(img)
    pil_img = numpy_arr_to_PIL_image(img, scale_to_255=False)
    pil_img.save(path)


def hstack_images(img1, img2):
    """
    Stacks 2 images side-by-side and creates one combined image.
    Args:
    - imgA: A numpy array of shape (M,N,3) representing rgb image
    - imgB: A numpy array of shape (D,E,3) representing rgb image
    Returns:
    - newImg: A numpy array of shape (max(M,D), N+E, 3)
    """

    # CHANGED
    imgA = np.array(img1)
    imgB = np.array(img2)
    Height = max(imgA.shape[0], imgB.shape[0])
    Width = imgA.shape[1] + imgB.shape[1]

    newImg = np.zeros((Height, Width, 3), dtype=imgA.dtype)
    newImg[: imgA.shape[0], : imgA.shape[1], :] = imgA
    newImg[: imgB.shape[0], imgA.shape[1] :, :] = imgB

    # newImg = PIL.Image.fromarray(np.uint8(newImg))
    return newImg

In [ ]:
# testing frames
import PIL
import numpy as np
p_img = PIL.Image.open()
def PIL_image_to_numpy_arr(img: Image, downscale_by_255: bool = True) -> np.ndarray:
    """
    Args:
        img: PIL Image
        downscale_by_255: whether to divide uint8 values by 255 to normalize
        values to range [0,1]
    Returns:
        img
    """
    img = np.asarray(img)
    img = img.astype(np.float32)
    if downscale_by_255:
        img /= 255
    return img

In [7]:
file_path = "2g1n6qdydwa9u22shpxqzp0t8m/P02/videos/"  # change to your own video path
files = os.listdir(file_path)
for f in files:
    path = file_path + f
    vid = cv2.VideoCapture(path)
    height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
    print(f'{f}: {height} x {width}')

P02_105.MP4: 1080.0 x 1920.0
P02_102.MP4: 1080.0 x 1920.0
P02_137.MP4: 1080.0 x 1920.0
P02_130.MP4: 1080.0 x 1920.0
P02_121.MP4: 1080.0 x 1920.0
P02_126.MP4: 1080.0 x 1920.0
P02_128.MP4: 1080.0 x 1920.0
P02_113.MP4: 1080.0 x 1920.0
P02_114.MP4: 1080.0 x 1920.0
P02_131.MP4: 1080.0 x 1920.0
P02_136.MP4: 1080.0 x 1920.0
P02_103.MP4: 1080.0 x 1920.0
P02_104.MP4: 1080.0 x 1920.0
P02_115.MP4: 1080.0 x 1920.0
P02_112.MP4: 1080.0 x 1920.0
P02_129.MP4: 1080.0 x 1920.0
P02_127.MP4: 1080.0 x 1920.0
P02_120.MP4: 1080.0 x 1920.0
P02_111.MP4: 1080.0 x 1920.0
P02_116.MP4: 1080.0 x 1920.0
P02_118.MP4: 1080.0 x 1920.0
P02_123.MP4: 1080.0 x 1920.0
P02_124.MP4: 1080.0 x 1920.0
P02_135.MP4: 1080.0 x 1920.0
P02_132.MP4: 1080.0 x 1920.0
P02_109.MP4: 1080.0 x 1920.0
P02_107.MP4: 1080.0 x 1920.0
P02_125.MP4: 1080.0 x 1920.0
P02_122.MP4: 1080.0 x 1920.0
P02_119.MP4: 1080.0 x 1920.0
P02_117.MP4: 1080.0 x 1920.0
P02_110.MP4: 1080.0 x 1920.0
P02_101.MP4: 1080.0 x 1920.0
P02_106.MP4: 1080.0 x 1920.0
P02_108.MP4: 1

## Narration embeddings

### Sentence BERT

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

c:\Users\barun\Miniconda3\envs\torch-zoo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentence = 'take knife and put back plate'
embedding = model.encode(sentence)
print(f'embedding shape = {embedding.shape}')
print(f'embedding = {embedding}')

embedding shape = (384,)
embedding = [-2.05573719e-02  5.83466925e-02 -3.61822210e-02 -3.85916838e-03
 -1.02967501e-01 -2.12387964e-02  1.13360612e-02  2.05004215e-02
 -1.21941715e-02 -5.22203930e-03  1.22582912e-01 -5.20467684e-02
 -8.58942643e-02  4.99771610e-02 -5.25188819e-02 -1.00123577e-01
  5.90349585e-02  1.52453454e-02  4.67252871e-03  3.19586881e-02
 -6.30533546e-02  9.30613652e-03  2.04242896e-02  5.70746977e-03
 -2.00125296e-02  8.88943598e-02 -1.95298449e-03  6.55974597e-02
 -2.26503033e-02 -1.94702987e-02 -1.39277820e-02 -5.86143062e-02
 -1.85706150e-02  9.50346701e-03 -1.17636397e-02  5.33422902e-02
 -3.14691029e-02  7.37754852e-02 -3.37130278e-02 -2.92711724e-02
 -3.96612473e-02  1.60753317e-02  4.74423803e-02 -2.67440602e-02
  8.83961916e-02  4.18313444e-02 -1.40461382e-02  2.41288580e-02
  9.60046723e-02 -3.51824276e-02 -2.10427269e-02  8.50031618e-03
 -2.69936658e-02 -8.40251446e-02  3.78844957e-03  5.67627512e-02
  2.86040478e-03  6.23897761e-02  4.86243144e-03  3.7

### BERT

Reference: https://towardsdatascience.com/3-types-of-contextualized-word-embeddings-from-bert-using-transfer-learning-81fcefe3fe6d

In [6]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\barun\Miniconda3\envs\torch-zoo\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\barun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 

In [7]:
def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS]" + text + "[SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensor = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensor

In [8]:
def get_bert_embeddings(tokens_tensor, segments_tensor, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensor)
        hidden_states = outputs[2][1:]

    token_embeddings = hidden_states[-1].squeeze()
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [15]:
texts = ['tap', 'tap:water', 'water:tap', 'flow:tap', 'tab', 'tap:hot']
word = 'tap'

In [16]:
target_word_embeddings = []
for text in texts:
    tokenized_text, tokens_tensor, segments_tensor = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensor, model)
    word_index = tokenized_text.index(word)
    word_embedding = list_token_embeddings[word_index]

    target_word_embeddings.append(word_embedding)

ValueError: 'tap' is not in list

Calcualte distance between the word embeddings

In [ ]:
from scipy.spatial.distance import cosine

list_of_distances = []
for text1, embed1 in zip(texts, target_word_embeddings):
    for text2, embed2 in zip(texts, target_word_embeddings):
        cos_dist = 1 - cosine(embed1, embed2)
        list_of_distances.append([text1, text2, cos_dist])

distances_df = pd.DataFrame(list_of_distances, 
                            columns=['text1', 'text2', 'cosine_distance'])
